## Part 1: Importing the libraries

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Dense, MaxPool2D, Flatten
import numpy as np 
tf.__version__

'2.2.0'

## Part 2: Data Preprocessing

In [2]:
#Preprocessing the training set 
#image augmentation in training set to avoid overfitting
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')                                                 

Found 8000 images belonging to 2 classes.


In [3]:
#Preprocessing the test set
test_datagen = ImageDataGenerator(rescale=1./255) #excluding other parameters as we do not want to manipulate test data
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary')                                                 

Found 2000 images belonging to 2 classes.


## Part 3: Building the CNN

In [19]:
#Initializing the CNN
cnn = Sequential()

**Step 1 - Convolution**

In [20]:
#third parameter in input shape stands for color scheme. 3 stands for RGB and 1 for grey scale images
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu',input_shape = (64,64,3))) 

**Step 2 - Pooling**

In [21]:
#third parameter in input shape stands for color scheme. 3 stands for RGB and 1 for grey scale images
cnn.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid')) 

**Adding a second convolutional layer**

In [22]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))  #input_shape parameter is only valid for first layer

**Step 3 - Flattening**

In [23]:
cnn.add(Flatten())

**Step 4 - Full Connection**

In [24]:
cnn.add(Dense(units=128,activation='relu'))

**Step 5 - Output Layer**

In [25]:
cnn.add(Dense(units=1,activation='sigmoid')) #for multiclass we have to use softmax function, for binary - sigmoid

## Part 4: Training the CNN model

In [26]:
#Compiling the model
cnn.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [30]:
#Training
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 422s 2s/step - loss: 0.6879 - accuracy: 0.5780 - val_loss: 0.6500 - val_accuracy: 0.6090
Epoch 2/25
250/250 [==============================] - 231s 924ms/step - loss: 0.6289 - accuracy: 0.6490 - val_loss: 0.7391 - val_accuracy: 0.5790
Epoch 3/25
250/250 [==============================] - 134s 537ms/step - loss: 0.5926 - accuracy: 0.6855 - val_loss: 0.5709 - val_accuracy: 0.7005
Epoch 4/25
250/250 [==============================] - 85s 340ms/step - loss: 0.5644 - accuracy: 0.7080 - val_loss: 0.5649 - val_accuracy: 0.7080
Epoch 5/25
250/250 [==============================] - 227s 909ms/step - loss: 0.5288 - accuracy: 0.7356 - val_loss: 0.5213 - val_accuracy: 0.7515
Epoch 6/25
250/250 [==============================] - 479s 2s/step - loss: 0.5121 - accuracy: 0.7483 - val_loss: 0.5014 - val_accuracy: 0.7560
Epoch 7/25
250/250 [==============================] - 629s 3s/step - loss: 0.4900 - accuracy: 0.7648 - val_loss: 0.5131 - val_accur

## Part 5: Making the predictions and evaluating the model

#### Predicting single new observation 1

In [33]:
#let us see the image for ourselves first and then pass it to the model for prediction
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "dataset/single_prediction/cat_or_dog_1.jpg",width=500,height=500)

In [36]:
test_image = load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64,64))
#convert the image pil format to an array for passing to predict method
test_image = img_to_array(test_image)

#images are sent to the model in a batch. Hence, we need to add this single image as first dimension of a batch
test_image = np.expand_dims(test_image, axis=0) #0 denotes first dimension i.e. first item in the batch
result = cnn.predict(test_image)
training_set.class_indices #will return what does 0 and 1 stand for (cat or dog)

{'cats': 0, 'dogs': 1}

In [37]:
if result[0][0] == 1:
    #first 0 indicates first batch and second 0 indicates the single element of the batch (i.e. the prediction)
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)

dog


#### Predicting single new observation 2

In [38]:
#Trying another picture downloaded from internet
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "dataset/single_prediction/cat_or_dog_3.jpg",width=500,height=500)

In [39]:
test_image = load_img('dataset/single_prediction/cat_or_dog_3.jpg', target_size = (64,64))
#convert the image pil format to an array for passing to predict method
test_image = img_to_array(test_image)

#images are sent to the model in a batch. Hence, we need to add this single image as first dimension of a batch
test_image = np.expand_dims(test_image, axis=0) #0 denotes first dimension i.e. first item in the batch
result = cnn.predict(test_image)
training_set.class_indices #will return what does 0 and 1 stand for (cat or dog)

if result[0][0] == 1:
    #first 0 indicates first batch and second 0 indicates the single element of the batch (i.e. the prediction)
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

dog


Our model could not predict the second picture correctly because the cat's face could hardly be seen

 **Note:** This code file was reproduced as a follow-along of the ML course - https://www.udemy.com/course/machinelearning/